# 7 - Sample Data Inheritance - The Microstructure Class 

**This sixth Notebook will introduce you to**:

1. the SampleData class inheritance mechanisms
2. The Microstructure class of the pymicro package
3. The different ways that exist to create a Microstructure object         
         
<div class="alert alert-info">

**Note** 
    
Throughout this notebook, it will be assumed that the reader is familiar with the overview of the SampleData file format and data model presented in the [first notebook of this User Guide](./SampleData_Introduction.ipynb) of this User Guide.

</div>

## SampleData Inheritance

The *SampleData* class implements a generic interface between users, numeric tools, and HDF5/XDMF multimodal datasets for material science and mechanics. It allows to create empty datasets or open existing ones, and leaves entirely to the user the definition of the dataset internal content and organization.

For specific and repeated applications, that always involve the same type of datasets, it may be convenient to standardize and predefine the internal organization of the dataset. For instance, to manage the data of a serie of material samples that are studied through SEM, EBSD imaging, and numerical simulation of the imaging digital twins, you will to define for each the same Image Group to store the imaging outputs, and a Mesh or Image group at least to store the simulation output. 

For such cases, it becomes convenient to define a more specific interface, for which all the internal organization of datasets (their data mode), is already defined. For that purpose, the *SampleData* class offers the possibility to create  inherited classes with a predefined data model through two particular and simple mechanisms, that are the subject of the present section.

### Custom Data Model

### Custom initialization

## The Microstructure Data Model

### A SampleData children class

### The CellData Group

### The MeshData Group 

### The GrainDataTable

### Phases

## Creating a Microstructure object

### From scratch

#### Empty Microstructure

#### Voronoi Microstructure

### From a data file

### From an existing Microstructure

#### Copy an existing Microstructure

#### Crop an existing Microstructure

## Tutorial Summary

In [ ]:
# code here